In [2]:
print("🎬 영화 추천 시스템 시작!")

import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# ✅ 환경 변수 로드
load_dotenv()

# ✅ Groq 모델 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# ✅ 단계 1: 장르 기반 영화 추천 프롬프트
prompt_recommend = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자가 좋아할 만한 영화를 추천해주는 AI입니다."),
    ("human", "{genre} 장르에서 추천할 만한 한국 영화를 한 편 알려주세요. 간단한 이유도 포함해주세요.")
])

# ✅ 단계 2: 영화 줄거리 요약 프롬프트
prompt_summary = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다."),
    ("human", "{movie} 영화의 줄거리를 3줄로 요약해 주세요. 문장마다 줄을 바꿔 주세요.")
])

# ✅ 파서 설정
parser = StrOutputParser()

# ✅ 체인 1: 장르 → 영화 추천
chain1 = prompt_recommend | llm | parser

# ✅ 체인 2: 영화 → 줄거리 요약
chain2 = (
    {"movie": chain1}
    | prompt_summary
    | llm
    | parser
)

# ✅ 입력
genre_input = "액션"

# ✅ 실행
try:
    print(f"\n[1단계] '{genre_input}' 장르 기반 영화 추천:")
    recommended_movie = chain1.invoke({"genre": genre_input})
    print(recommended_movie)

    print("\n[2단계] 추천 영화 줄거리 요약:")
    result_summary = chain2.invoke({"genre": genre_input})  # genre는 chain1에 전달됨
    print(result_summary)

except Exception as e:
    print(f"❌ 오류 발생: {e}")


🎬 영화 추천 시스템 시작!

[1단계] '액션' 장르 기반 영화 추천:
국내에서 제작된 액션 영화 중 추천할 만한 작품은 '부산행'(2016)입니다. 이 영화는 좀비 바이러스가 확산되는 상황에서 가족과 기차에 탄 승객들이 살아남기 위해 싸우는 내용을 담고 있습니다. 특히, 배우 정재광, 마동석, 손석구 등 다양한 출연진이 등장하여 흥미로운 장면을 선사합니다. 또한, 긴박한 상황 속에서 인물들이 보여주는 액션과 감동적인 연기는 시청자들에게 큰 인상을 남길 것입니다.

[2단계] 추천 영화 줄거리 요약:
영화 '부산행'은 서울에서 부산으로 향하는 KTX 열차에서 시작됩니다.

열차 안에서 갑자기 발생한 좀비 바이러스에 의해 탑승객들이 하나둘 감염되기 시작합니다.

주인공들은 생존을 위해 열차에서 탈출하고, 좀비들과의 사투를 벌이며 부산행의 여정을 계속하게 됩니다.
